# Pydantic fundamentals

## Start without Pydantic

In [2]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name = "Fredrik", age = 30, gender = "M")
person1

In [3]:
person1.name, person1.age

('Fredrik', 30)

In [4]:
person2 = Person(name = 3213+3213, gender = True, age = "minus ten")
person2

## Validation of our Person class

In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be of type string, not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

try:
    Person(name = 3.1415, age = 30, gender = "M")
except TypeError as err:
    print(err)

Name must be of type string, not <class 'float'>


In [9]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be of type string, not {type(name)}")
        self.name = name
        self.gender = gender

        if not isinstance(age, int):
            raise TypeError(f"Age must be of type int, not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"Age must be between 0 and 124, not {age}")
        self.age = age

    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"

try:
    person3 = Person(name = "Ragnar", age = -35, gender = "M")
except ValueError as err:
    print(err)

Age must be between 0 and 124, not -35


In [10]:
person4 = Person(name = "Bella", age = 4, gender = "F")
person4

Person(Bella, F, 4)

In [11]:
person4.age = -5
person4

Person(Bella, F, -5)

In [16]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be of type string, not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if not isinstance(age, int):
            raise TypeError(f"Age must be of type int, not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"Age must be between 0 and 124, not {age}")
        self._age = age

    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"
    
person5 = Person(name = "Natalie", age = 4, gender = "F")
person5

Person(Natalie, F, 4)

In [19]:
try:
    person5.age = 125
except ValueError as err:
    print(err)

Age must be between 0 and 124, not 125


In [22]:
try:
    person6 = Person(name = "Natalie", age = "4", gender = "F")
except TypeError as err:
    print(err)

Age must be of type int, not <class 'str'>


## Validate using Pydantic

In [28]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person7 = Person(name = "Christina", gender = "F", age = 29)
person7

Person(name='Christina', gender='F', age=29)

In [29]:
person7.age = 33
person7

Person(name='Christina', gender='F', age=33)

In [30]:
# issue
person7.age = "thirty"
person7

Person(name='Christina', gender='F', age='thirty')

In [31]:
from pydantic import ValidationError

try:
    person7 = Person(name = 940801, gender = "F", age = 29)
except ValidationError as err:
    print(err)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=940801, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:
# str "29" is coerced into int 29
Person(name = "Nina", gender = "F", age = "29")

Person(name='Nina', gender='F', age=29)

In [37]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int

    model_config = ConfigDict(validate_assignment=True)

person8 = Person(name = "Nina", gender = "F", age = "29")

try:
    person8.age = "thirty"
except ValidationError as err:
    print(err)

person8.age = 10
person8

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person(name='Nina', gender='F', age=10)

In [38]:
try:
    person7 = Person(name = 3.1415, gender = 2.714, age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=3.1415, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [39]:
try:
    person7 = Person(name = "Sofia", gender = "F", age = -4)
except ValidationError as err:
    print(err)

## Add age validation

In [42]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt = -1, lt = 125)

    model_config = ConfigDict(validate_assignment=True)

try:
    Person(name = "Sofia", gender = "Fem", age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serialization and deserialization

In [46]:
# serialization
person8.model_dump()

{'name': 'Nina', 'gender': 'F', 'age': 10}

In [48]:
import json
with open("person.json", "w") as file:
    json.dump(person8.model_dump(), file)

In [50]:
with open("person.json", "r") as file:
    json_data = file.read()

print(repr(json_data))

# deserialization
Person.model_validate_json(json_data)

'{"name": "Nina", "gender": "F", "age": 10}'


Person(name='Nina', gender='F', age=10)